# <div align="center"><b> Fixes aplicados </b></div>

<div align="right">

<!-- [![Binder](http://mybinder.org/badge.svg)](https://mybinder.org/) -->
[![nbviewer](https://img.shields.io/badge/render-nbviewer-orange?logo=Jupyter)](https://nbviewer.org/)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/)

</div>

* * *

<style>
/* Limitar la altura de las celdas de salida en html */
.jp-OutputArea.jp-Cell-outputArea {
    max-height: 500px;
}
</style>

🛻 <em><font color='MediumSeaGreen'>  Instalaciones: </font></em> 🛻


Este notebook utiliza [Poetry](https://python-poetry.org/) para la gestión de dependencias.
Primero instala Poetry siguiendo las instrucciones de su [documentación oficial](https://python-poetry.org/docs/#installation).
Luego ejecuta el siguiente comando para instalar las dependencias necesarias y activar el entorno virtual:

- Bash:

```bash
poetry install
eval $(poetry env activate)
```

- PowerShell:

```powershell
poetry install
Invoke-Expression (poetry env activate)
```

> 📝 <em><font color='Gray'>Nota:</font></em> Para agregar `pytorch` utilizando Poetry, se utiliza el siguiente comando:
> ```bash
> # Más info: https://github.com/python-poetry/poetry/issues/6409
> potery source add --priority explicit pytorch_gpu https://download.pytorch.org/whl/cu128 # Seleccionar la wheel adecuada para tu GPU
> poetry add --source pytorch_gpu torch torchvision 
> ```

✋ <em><font color='DodgerBlue'>Importaciones:</font></em> ✋

In [ ]:
# Recarga automática de módulos en Jupyter Notebook
%reload_ext autoreload
%autoreload 2

import sys, datetime, os
from pprint import pprint

from modulo_apps.config import config as CONFIG
from loguru import logger as LOGGER
from modulo_apps.utils.helpers import set_log_to_file

import cv2

from modulo_apps.database_comunication.mongodb_client import mongodb as DB
from modulo_apps.s3_comunication.s3_client import s3client as S3


🔧 <em><font color='tomato'>Configuraciones:</font></em> 🔧


In [2]:
LOGGER.info("Configuración cargada correctamente.")

2025-08-09 16:13:04.308 | INFO     | __main__:<module>:1 - Configuración cargada correctamente.


<div align="center">✨Datos del proyecto:✨</div>

<p></p>

<div align="center">

| Subtitulo       | Arreglo de los datos                                                                       |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Descrpción**  | <small>Este notebook regstra los arreglos de los datos que se hicieron durante el proyecto. Impacta principalmente MongoDB pero también MinIO</small>|

</div>

## Pruebas de conexión

In [8]:
try:
    # Verificar conexión a MongoDB
    DB.list_collections()
    LOGGER.info("Conexión a MongoDB exitosa.")
except Exception as e:
    LOGGER.error(f"Error al conectar a MongoDB: {e}")

try:
    # Verificar conexión a S3
    S3.list_buckets()
    LOGGER.info("Conexión a S3 exitosa.")
except Exception as e:
    LOGGER.error(f"Error al conectar a S3: {e}")

2025-08-09 16:17:18.612 | INFO     | __main__:<module>:4 - Conexión a MongoDB exitosa.
2025-08-09 16:17:18.621 | INFO     | __main__:<module>:11 - Conexión a S3 exitosa.


## Fixes aplicados

In [ ]:
def fix_image_width_height(image_id: str, width: int, height: int) -> bool:
    """Actualiza el ancho y alto de una imagen en la base de datos.

    Args:
        image_id (str): Identificador de la imagen a actualizar.
        width (int): Nuevo ancho de la imagen.
        height (int): Nuevo alto de la imagen.

    Returns:
        bool: True si la actualización fue exitosa, False en caso contrario.
    """
    try:
        imagenes = DB.get_collection("imagenes")
        result = imagenes.update_one(
            {"id": image_id},
            {
                "$set": {
                    "width": width,
                    "height": height,
                }
            },
        )
        if result.modified_count > 0:
            LOGGER.debug(f"Ancho y alto de la imagen {image_id} actualizados correctamente.")
            return True
        else:
            LOGGER.warning(f"No se encontraron cambios para la imagen {image_id}.")
            return False
    except Exception as e:
        LOGGER.error(f"Error al actualizar el ancho y alto de la imagen {image_id}: {e}")
        return False

In [ ]:
def fix_width_height_images():
    """Corrige el ancho y alto de las imágenes en la base de datos.

    Esta función descarga las imágenes desde MinIO, obtiene sus dimensiones (ancho y alto),
    actualiza estos valores en la base de datos y elimina las imágenes descargadas localmente.

    Pasos realizados:
    1. Configura el logger para registrar las actualizaciones.
    2. Obtiene todas las imágenes almacenadas en la base de datos.
    3. Descarga cada imagen desde MinIO si está marcada como descargada.
    4. Calcula las dimensiones de la imagen descargada.
    5. Actualiza las dimensiones en la base de datos.
    6. Elimina la imagen descargada localmente.

    Raises:
        OSError: Si ocurre un error al eliminar la imagen descargada localmente.
    """

    set_log_to_file(f"updates_{datetime.date.today()}.log", level="INFO")
    LOGGER.info("Iniciando actualización de imágenes y parches.")
    # Actualizar el width y height de las imágenes en la base de datos
    # 1 - Obtener todas las imágenes de la base de datos
    imagenes = DB.get_collection("imagenes")

    # 2 - Para cada imagen:
    for image in imagenes.find():
        image_id = image["id"]
        downloaded: bool = image["downloaded"]
        if not downloaded:
            LOGGER.warning(f"La imagen {image_id} no está descargada, se omitirá.")
            continue
        LOGGER.info(f"Actualizando imagen {image_id}...")
        # 2.1 - Descargar la imagen desde MinIO

        jpg_path = download_image_from_minio(image_id)
        LOGGER.info(f"Imagen {image_id} descargada correctamente.")
        # 2.2 - Obtener el ancho y alto de la imagen descargada
        img = cv.imread(jpg_path)
        height, width = img.shape[:2]
        # 2.3 - Actualizar el ancho y alto de la imagen en la base de datos
        fix_image_width_height(image_id, width, height)
        LOGGER.info(f"Ancho y alto de la imagen {image_id} actualizados correctamente.")
        # 2.4 - Eliminar la imagen descargada
        try:
            os.remove(jpg_path)
            LOGGER.info(f"Imagen {jpg_path} eliminada correctamente.")
        except OSError as e:
            LOGGER.warning(f"Error al eliminar la imagen {jpg_path}: {e}")
            continue
    LOGGER.info("Actualización de imágenes y parches finalizada.")

In [ ]:
def fix_dates_images():
    set_log_to_file(f"fix_dates_images_{datetime.date.today()}.log")
    LOGGER.setLevel(logging.INFO)
    LOGGER.info("Iniciando actualización de fechas de imágenes.")
    # Actualizar el width y height de las imágenes en la base de datos
    # 1 - Obtener todas las imágenes de la base de datos
    imagenes = DB.get_collection("imagenes")
    date_format = "%d/%m/%Y"

    # 2 - Para cada imagen:
    for image in imagenes.find():
        # 2.1 - Obtener el título de la imagen
        image_title = image["title"]

        # 2.2 - Extraer la fecha del título de la imagen
        match = re.search(r"\d{2}/\d{2}/\d{4}", image_title)
        date_str = match.group(0) if match else None
        if not date_str:
            LOGGER.warning(f"No se encontró una fecha válida en el título de la imagen {image_title}.")
            continue

        # 2.3 - Actualizar la fecha de captura en la base de datos
        try:
            date_captured = datetime.datetime.strptime(date_str, date_format)
            imagenes.update_one(
                {"id": image["id"]},
                {
                    "$set": {
                        "date_captured": date_captured,
                    }
                },
            )
            LOGGER.info(f"Fecha de captura de la imagen {image_title} actualizada correctamente.")
        except Exception as e:
            LOGGER.warning(f"Error al actualizar la fecha de captura: {e}")

    LOGGER.info("Actualización de fechas de imágenes finalizada.")

In [ ]:
def fix_patch_name():
    """Corrige el nombre de los parches en la base de datos.
    Le saca el .jpg al final del nombre del parche y lo actualiza en la base de datos.
    """
    set_log_to_file(f"fix_patch_name_{datetime.date.today()}.log")
    LOGGER.setLevel(logging.INFO)
    LOGGER.info("Iniciando actualización de nombres de parches.")
    # 1 - Obtener todas las imágenes de la base de datos
    imagenes = DB.get_collection("imagenes")

    # 2 - Para cada imagen:
    for image in imagenes.find():
        image_id = image["id"]
        # 2.1 - Obtener los parches de la imagen
        try:
            patches = image["patches"]
        except KeyError:
            LOGGER.warning(f"No se encontraron parches para la imagen {image_id}.")
            continue
        # 2.2 - Para cada parche:
        for patch in patches:
            patch_name = patch["patch_name"]
            if patch_name.endswith(".jpg"):
                new_patch_name = patch_name[:-4]
                # 2.3 - Actualizar el nombre del parche en la base de datos
                imagenes.update_one(
                    {"id": image_id, "patches.patch_name": patch_name},
                    {"$set": {"patches.$.patch_name": new_patch_name}},
                )
                LOGGER.info(f"Nombre del parche {patch_name} actualizado a {new_patch_name}.")
            else:
                LOGGER.warning(f"El parche {patch_name} no tiene .jpg al final.")
    LOGGER.info("Actualización de nombres de parches finalizada.")